In [4]:
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
import requests

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ticker = yf.Ticker('2445.KL')

In [9]:
pd.DataFrame(ticker.quarterly_financials['2025-06-30'])

,2025-06-30
Tax Effect Of Unusual Items,7.331187e+05
Tax Rate For Calcs,2.615480e-01
Normalized EBITDA,9.081600e+08
Total Unusual Items,2.803000e+06
Total Unusual Items Excluding Goodwill,2.803000e+06
Net Income From Continuing Operation Net Minority Interest,3.465940e+08
Reconciled Depreciation,2.616150e+08
EBITDA,9.109630e+08
EBIT,6.493480e+08
Net Interest Income,-1.239910e+08


In [10]:
import plotly.graph_objects as go

In [11]:
labels = [
    "Total Revenue",
    "Operating Expenses",
    "Other Operating Income",
    "Operating Income",
    "Finance Costs",
    "Pretax Income",
    "Taxation",
    "Net Income"
]

In [17]:
import plotly.graph_objects as go

# --- Node labels (readable)
labels = [
    # Segment revenues
    "Plantation (derived)", "Manufacturing", "Property Development",
    # Aggregation layer
    "Total Revenue",
    # revenue split
    "Cost of Sales / Operating expenses", "Gross before Other Income",
    # other operating income flows into operating income
    "Other operating income",
    "Operating Income",
    # Non-operating adjustments (small)
    "Net non-operating items (rev/jv/impairment, net)",
    "Finance costs",
    "Profit before taxation",
    "Taxation",
    "Net profit (period)"
]

# --- Indices (for readability)
I_PLANT = labels.index("Plantation (derived)")
I_MANU  = labels.index("Manufacturing")
I_PROP  = labels.index("Property Development")
I_REV   = labels.index("Total Revenue")
I_COST  = labels.index("Cost of Sales / Operating expenses")
I_GROSS = labels.index("Gross before Other Income")
I_OTHER = labels.index("Other operating income")
I_OPINC = labels.index("Operating Income")
I_NONOP = labels.index("Net non-operating items (rev/jv/impairment, net)")
I_FC   = labels.index("Finance costs")
I_PBT  = labels.index("Profit before taxation")
I_TAX  = labels.index("Taxation")
I_NP   = labels.index("Net profit (period)")

# --- Values (RM '000) — from PDF (and derived plantation)
values_map = {
    "plantation_rev": 981_740,      # derived residual
    "manufacturing_rev": 5_384_000, # stated in report text
    "property_rev": 66_600,         # stated in report text
    "revenue_total": 6_432_340,     # statement
    "operating_expenses": 5_861_688,
    "other_oper_income": 77_470,
    "gross_before_other": 6_432_340 - 5_861_688,  # 570,652
    "operating_income": 648_122,    # statement
    "net_nonop": 1_226,             # 2,803 + 771 - 2,348
    "finance_costs": 123_991,
    "pbt": 525_357,
    "tax": 137_406,
    "net_profit": 387_951
}

# --- Links (source indices, target indices, values)
sources = [
    # segments -> Total Revenue
    I_PLANT, I_MANU, I_PROP,
    # Total Revenue -> Cost and Gross
    I_REV, I_REV,
    # Gross (remainder) -> Operating Income
    I_GROSS,
    # Other operating income -> Operating Income
    I_OTHER,
    # Operating Income -> Finance costs (expense)
    I_OPINC,
    # Operating Income -> Profit before tax (main flow)
    I_OPINC,
    # Net non-op items -> Profit before tax (small add)
    I_NONOP,
    # Profit before tax -> Tax, Net profit
    I_PBT, I_PBT
]

targets = [
    # segments -> Total Revenue
    I_REV, I_REV, I_REV,
    # Total Revenue -> Cost and Gross
    I_COST, I_GROSS,
    # Gross -> Operating Income
    I_OPINC,
    # Other operating income -> Operating Income
    I_OPINC,
    # Operating Income -> Finance costs (shows the cost being paid out of OP inc)
    I_FC,
    # Operating Income -> Profit before tax
    I_PBT,
    # Net non-op -> Profit before tax
    I_PBT,
    # Profit before tax -> Tax and Net profit
    I_TAX, I_NP
]

values = [
    # segment -> Revenue
    values_map["plantation_rev"],
    values_map["manufacturing_rev"],
    values_map["property_rev"],
    # revenue -> cost & gross
    values_map["operating_expenses"],
    values_map["gross_before_other"],
    # gross -> operating income (gross remainder becomes operating income after other income)
    values_map["gross_before_other"],
    # other operating income -> operating income
    values_map["other_oper_income"],
    # operating income -> finance costs (expense)
    values_map["finance_costs"],
    # operating income -> profit before tax (main)
    values_map["pbt"],  # note: pbt reflects op income minus finance costs plus small net non-op
    # net non-op -> profit before tax
    values_map["net_nonop"],
    # profit before tax -> tax and net profit
    values_map["tax"],
    values_map["net_profit"]
]

# Note: some flows intentionally appear twice (e.g. gross_before_other used both as
# Revenue->Gross and Gross->Operating Income) to keep the visual layering clear.

# --- Create Sankey
fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=labels,
        pad=18,
        thickness=18,
        line=dict(color="black", width=0.5),
        color=[
            "#c7e9c0", "#7fcdbb", "#41b6c4",    # revenue segments
            "#feb24c",                          # revenue aggregate
            "#f03b20", "#fdae6b",               # cost & gross
            "#a1d99b", "#74c476",               # other income / op inc
            "#bdbdbd", "#f16913",               # non-op, finance cost
            "#2b8cbe", "#6baed6", "#9ecae1"     # pbt, tax, net profit
        ]
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        # subtle translucent links
        color=["rgba(0,0,0,0.15)"] * len(values)
    )
)])

fig.update_layout(
    title_text="KLK (2445.KL) — Q3 (3 months ended 30 Jun 2025) Financial Flow (RM'000) — Source: KLK Interim Report",
    font=dict(size=11)
)

fig.write_html("klk_sankey.html", auto_open=True)

In [18]:
import sqlite3

data = [
    ("2025Q3", "Plantation (derived)", "Total Revenue", 981740, "Residual segment revenue"),
    ("2025Q3", "Manufacturing", "Total Revenue", 5384000, "Manufacturing segment"),
    ("2025Q3", "Property Development", "Total Revenue", 66600, "Property segment"),
    ("2025Q3", "Total Revenue", "Cost of Sales / Operating expenses", 5861688, "Operating expenses (cost of sales)"),
    ("2025Q3", "Total Revenue", "Gross before Other Income", 570652, "Derived: revenue − cost"),
    ("2025Q3", "Gross before Other Income", "Operating Income", 570652, "Gross profit portion"),
    ("2025Q3", "Other operating income", "Operating Income", 77470, "Other operating income"),
    ("2025Q3", "Operating Income", "Finance costs", 123991, "Finance costs (expense)"),
    ("2025Q3", "Operating Income", "Profit before taxation", 525357, "Operating profit after finance costs"),
    ("2025Q3", "Net non-operating items (rev/jv/impairment, net)", "Profit before taxation", 1226, "Net non-operating gains/losses"),
    ("2025Q3", "Profit before taxation", "Taxation", 137406, "Tax provision"),
    ("2025Q3", "Profit before taxation", "Net profit (period)", 387951, "Net profit after tax")
]

conn = sqlite3.connect("src/data/bursa_palmai_database.db")
cur = conn.cursor()




OperationalError: unable to open database file

In [10]:
import requests
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
import sqlite3
import os
import numpy as np

In [ ]:
# === 1. Fetch weather forecast ===
response = requests.get("https://api.data.gov.my/weather/forecast")
wfcast_json = response.json()
wfcast_df = pd.json_normalize(wfcast_json)

wfcast_df = wfcast_df[['date', 'summary_forecast', 'min_temp', 'max_temp', 'location.location_name']]
wfcast_df.rename(columns={'location.location_name': 'location_name'}, inplace=True)
wfcast_df['date'] = pd.to_datetime(wfcast_df['date'])

# === 2. Load weather station coordinates ===
BASE_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(BASE_DIR, ".."))  # go up one level
DATA_PATH = os.path.join(PROJECT_ROOT, "src", "data", "weather_station_base.csv")

points_df = pd.read_csv(DATA_PATH)

# === 3. Merge weather with station coordinates ===
weather_df = wfcast_df.merge(points_df, on='location_name', how='left')
weather_gdf = gpd.GeoDataFrame(
    weather_df,
    geometry=gpd.points_from_xy(weather_df.base_longitude, weather_df.base_latitude),
    crs="EPSG:4326"
)

# === 4. Load MSPO Certified Entities from SQLite ===
BASE_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(BASE_DIR, ".."))  # go up one level
DB_PATH = os.path.join(PROJECT_ROOT, "src", "data", "bursa_palmai_database.db")
conn = sqlite3.connect(DB_PATH)

mspo_df = pd.read_sql_query("""
    SELECT
        "company" AS company_name,
        "parent_company" AS parent_company,
        "latitude" AS latitude,
        "longitude" AS longitude,
        "certified_area_ha" AS certified_area,
        "planted_area_ha" AS planted_area
    FROM mspo_certified_entities
    WHERE status = 'ACTIVE'
        AND state = "Pahang"
""", conn)
conn.close()

for col in ['certified_area', 'planted_area']:
    mspo_df[col] = (
        pd.to_numeric(
            mspo_df[col]
            .replace(['', ' ', '-', None, 'NA', 'N/A'], np.nan),  #type: ignore
            errors='coerce'  # safely convert any invalid to NaN
        )
    )

# Add certified area percentage (handle zero and missing values safely)
mspo_df['certified_area_pct'] = (
    mspo_df['certified_area'] /
    mspo_df['planted_area'].replace(0, np.nan)
) * 100

mspo_df = mspo_df.dropna(subset=['certified_area_pct'])
mspo_df['certified_area_pct'] = mspo_df['certified_area_pct'].round(2)

# === 5. Convert to GeoDataFrame (points) ===
mspo_gdf = gpd.GeoDataFrame(
    mspo_df,
    geometry=gpd.points_from_xy(mspo_df.longitude, mspo_df.latitude),
    crs="EPSG:4326"
)

# === 6. Prepare station GeoDataFrame ===
# === 6. Prepare station GeoDataFrame safely ===
station_points = weather_gdf[['location_name', 'base_longitude', 'base_latitude']].drop_duplicates()

# ensure geometry has same length as station_points, not weather_gdf
station_points['geometry'] = gpd.points_from_xy(
    station_points['base_longitude'],
    station_points['base_latitude']
)

station_gdf = gpd.GeoDataFrame(station_points, geometry='geometry', crs='EPSG:4326')

# === 7. Find nearest weather station for each plantation ===
def get_nearest_station(row):
    plantation_point = (row['latitude'], row['longitude'])
    distances = station_gdf.apply(
        lambda x: geodesic(plantation_point, (x.base_latitude, x.base_longitude)).km,
        axis=1
    )
    idx = distances.idxmin()
    nearest = station_gdf.loc[idx]
    return pd.Series({
        'nearest_station': nearest.location_name,
        'distance_km': round(distances[idx], 2)
    })

mspo_gdf[['nearest_station', 'distance_km']] = mspo_gdf.apply(get_nearest_station, axis=1)

# === 8. Merge weather forecast ===
mspo_forecast = pd.merge(
    mspo_gdf,
    weather_gdf,
    left_on='nearest_station',
    right_on='location_name',
    how='left'
)

# === 9. Final cleanup ===
mspo_forecast = mspo_forecast[
    ['company_name', 'parent_company', 'latitude', 'longitude',
     'certified_area', 'planted_area', 'certified_area_pct',
     'nearest_station', 'distance_km', 'summary_forecast',
     'min_temp', 'max_temp', 'date']
].drop_duplicates()

print("✅ Forecast mapping complete!")
print(mspo_forecast.head())

C:\Users\Dell\AppData\Local\Temp\ipykernel_8176\2063166911.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['', ' ', '-', None, 'NA', 'N/A'], np.nan),  # unify missing as np.nan


✅ Forecast mapping complete!
             company_name          parent_company  latitude   longitude  \
0  PKNP AGRO TECH SDN BHD  PKNP AGRO TECH SDN BHD  3.593309  103.288625   
1  PKNP AGRO TECH SDN BHD  PKNP AGRO TECH SDN BHD  3.593309  103.288625   
2  PKNP AGRO TECH SDN BHD  PKNP AGRO TECH SDN BHD  3.593309  103.288625   
3  PKNP AGRO TECH SDN BHD  PKNP AGRO TECH SDN BHD  3.593309  103.288625   
4  PKNP AGRO TECH SDN BHD  PKNP AGRO TECH SDN BHD  3.593309  103.288625   

   certified_area  planted_area  certified_area_pct nearest_station  \
0          1922.0        1922.0               100.0           Pekan   
1          1922.0        1922.0               100.0           Pekan   
2          1922.0        1922.0               100.0           Pekan   
3          1922.0        1922.0               100.0           Pekan   
4          1922.0        1922.0               100.0           Pekan   

   distance_km                summary_forecast  min_temp  max_temp       date  
0        15.2

In [18]:
mspo_forecast.shape

(2440, 13)

In [5]:
def get_fertilizer_data():
    commodities = [
        "urea", "triple-superphosphate", "rock-phosphate",
        "potassium-chloride", "dap-fertilizer"
    ]
    all_data = []

    for item in commodities:
        url = f"https://www.indexmundi.com/commodities/?commodity={item}&months=30&currency=myr"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", id="gvPrices")
        
        if table is None:
            print(f"Table not found for {item} — skipping")
            continue
        
        rows = table.find_all("tr")[1:]
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 2:
                all_data.append({
                    "Month": cols[0].text.strip(),
                    "Price": cols[1].text.strip().replace(",", ""),
                    "Commodity": item
                })

    df = pd.DataFrame(all_data)
    df["Month"] = pd.to_datetime("01 " + df["Month"], format="%d %b %Y", errors="coerce")
    df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
    df = df.dropna()

    pivot_df = df.pivot_table(index="Month", columns="Commodity", values="Price").reset_index()
    pivot_df["Month"] = pivot_df["Month"].dt.strftime("%Y-%m")

    return pivot_df.to_dict(orient="list")

In [7]:
pd.DataFrame(get_fertilizer_data())

,Month,dap-fertilizer,potassium-chloride,rock-phosphate,triple-superphosphate,urea
0,2023-03,2707.96,2024.27,1541.66,2401.86,1400.90
1,2023-04,2817.26,1802.25,1525.83,2432.48,1385.98
2,2023-05,2310.46,1687.54,1562.96,2200.06,1491.61
3,2023-06,2107.23,1520.56,1597.05,1807.99,1332.81
4,2023-07,2104.94,1565.80,1571.54,1799.81,1535.42
5,2023-08,2436.49,1627.23,1595.53,2076.51,1776.99
6,2023-09,2470.58,1647.36,1626.30,2159.82,1778.40
7,2023-10,2538.56,1622.97,1649.65,2222.30,1952.90
8,2023-11,2511.28,1556.01,1629.24,2169.02,1807.40
9,2023-12,2632.71,1456.48,712.18,2089.83,1653.18
